In [10]:
import torch
from torch import nn
from torch import optim
import torchvision

from tqdm import tqdm_notebook

In [2]:
kind_and_index = [
    '飞机',
    '汽车',
    '小鸟',
    '猫咪',
    '小鹿',
    '小狗',
    '青蛙',
    '小马',
    '船舶',
    '卡车'
]

In [ ]:
cifar_10_train = torchvision.datasets.CIFAR10('../01_data/', download=True, transform=torchvision.transforms.ToTensor())
cifar_10_test = torchvision.datasets.CIFAR10('../01_data/', download=True, train=False, transform=torchvision.transforms.ToTensor())

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size=16,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=16,
                                          shuffle=False)

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=(3, 3))
        self.fc1 = nn.Linear(in_features=8*8*8, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=10)

    def forward(self, x):
        in_size = x.size(0)
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)
        out = self.conv2(out)
        out = F.relu(out)
        out = out.view(in_size, -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = torch.log_softmax(out, dim=1)
        return out

In [8]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
def train(model: ConvNet, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        running_loss = 0.0
        running_corrects = 0
        for data, targets in tqdm_notebook(train_loader):
            # 1. 清空优化器梯度
            optimizer.zero_grad()
            # 2. 读入data，label，并进行形状变换
#             inputs = inputs.view(-1, 32 * 32 * 3)     
            # 3. 运行模型前向传播过程
            outputs = model(data)
            # 4. 基于模型输出生成最终结果
            _, preds = torch.max(outputs, 1)
            # 5. 计算损失
            loss = criterion(outputs, targets)
            # 6. 基于损失计算梯度
            loss.backward()
            # 7. 基于梯度更新参数
            optimizer.step()
            
            # 一个批次数据的损失函数的计算
            running_loss += loss.items() * inputs.size(0)
            # 一个批次数据准确率的计算
            running_corrects += torch.sum(preds == labels.data)

In [12]:
def test():
    pass


In [ ]:
train(model, train_loader)